In [1]:
import pandas as pd
import numpy as np
import string
from string import punctuation
from tensorflow.keras.layers import LSTM, Dense, Embedding, Input
from keras.models import Model
from tensorflow.keras.models import Model
import re
from string import digits
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/prismspeechproject/neural/master/implementation/English_hindi.csv', encoding='utf-8')
df = df[df.columns[0:2]]
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,8,9,10,11,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,English,Hindi
0,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


In [3]:
# df.rename(columns={'english_sentence' : 'English', 'hindi_senten'}, inplace=True)
# df.columns

In [4]:
df = df[~pd.isnull(df['English'])]
df.shape

(127605, 2)

In [5]:
df.drop_duplicates(subset="English", keep='first' , inplace=True)
print(df.shape)
df.head(10)

(124303, 2)


,English,Hindi
0,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।
5,The then Governor of Kashmir resisted transfer...,कश्मीर के तत्कालीन गवर्नर ने इस हस्तांतरण का व...
6,In this lies the circumstances of people befor...,इसमें तुमसे पूर्व गुज़रे हुए लोगों के हालात हैं।
7,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
8,“”Global Warming“” refer to warming caused in ...,ग्लोबल वॉर्मिंग से आशय हाल ही के दशकों में हुई...
9,You may want your child to go to a school that...,हो सकता है कि आप चाहते हों कि आप का नऋर्नमेनटे...


In [6]:
punctuations = set(string.punctuation)
rm_digits = str.maketrans('', '', digits)
df = df.sample(n=30000, random_state=42)
# typing to str
df['English'] = df['English'].apply(lambda e: str(e))
df['Hindi'] = df['Hindi'].apply(lambda h: str(h))

#Here all the data inside () or [] are removed
df['English'] = df['English'].apply(lambda e: re.sub(r"[\(\[].*?[\)\]]", '', e))
df['Hindi'] = df['Hindi'].apply(lambda h : re.sub(r"[\(\[].*?[\)\]]", '', h))

#removing punctuations 
df['English'] = df['English'].apply(lambda e: ''.join(c for c in e if c not in punctuations))
df['Hindi'] = df['Hindi'].apply(lambda h: ''.join(c for c in h if c not in punctuations))

#lower
df['English'] = df['English'].apply(lambda e: e.lower())
df['Hindi'] = df['Hindi'].apply(lambda h: h.lower())

#Remove digits
df['English']=df['English'].apply(lambda e: e.translate(rm_digits))
df['Hindi']=df['Hindi'].apply(lambda h: h.translate(rm_digits))
df['Hindi']=df['Hindi'].apply(lambda h: re.sub("[a-z२३०८१५७९४६]", '', h))

#Striping
df['English'] = df['English'].apply(lambda e: e.strip())
df['Hindi'] = df['Hindi'].apply(lambda h: h.strip())

df.head()

,English,Hindi
113416,they were to be smuggled through the northeast...,इन हथियारों की उतरपूर्वी सरहदों से चोरी से लाय...
68207,by holding the begging bowl and receiving alms,भीख के कटोरे में दान लेकर
56337,this technology the women will not share with ...,इस तकनीक को औरतें मर्दों को नहीं बताती हैं।
81595,and the way that the reality is,इस तरह की चित्रकारी के असली महत्व में।
105613,we can do to the next harvest,हम लोग अगली फसल तक ये कर सकते है


In [7]:
#Appending Staring and Ending notation in Hindi column
df['Hindi'] = df['Hindi'].apply(lambda h: 'START_ ' + h + ' _END')
df.head()

,English,Hindi
113416,they were to be smuggled through the northeast...,START_ इन हथियारों की उतरपूर्वी सरहदों से चोरी...
68207,by holding the begging bowl and receiving alms,START_ भीख के कटोरे में दान लेकर _END
56337,this technology the women will not share with ...,START_ इस तकनीक को औरतें मर्दों को नहीं बताती ...
81595,and the way that the reality is,START_ इस तरह की चित्रकारी के असली महत्व में। ...
105613,we can do to the next harvest,START_ हम लोग अगली फसल तक ये कर सकते है _END


In [8]:
# extract words
eng_words = []
for sen in df['English']:
  for w in sen.split():
    if w not in eng_words:
      eng_words.append(w)
hin_words = []
for sen in df['Hindi']:
  for w in sen.split():
    if w not in hin_words:
      hin_words.append(w)
len(eng_words), len(hin_words)

(33614, 37769)

In [9]:
# Calculate number of words in each line of 2 columns
df['len_eng_words'] = df['English'].apply(lambda e: len(e.split(" ")))
df['len_hin_words'] = df['Hindi'].apply(lambda h: len(h.split(" ")))
df.head()

,English,Hindi,len_eng_words,len_hin_words
113416,they were to be smuggled through the northeast...,START_ इन हथियारों की उतरपूर्वी सरहदों से चोरी...,19,20
68207,by holding the begging bowl and receiving alms,START_ भीख के कटोरे में दान लेकर _END,8,8
56337,this technology the women will not share with ...,START_ इस तकनीक को औरतें मर्दों को नहीं बताती ...,10,11
81595,and the way that the reality is,START_ इस तरह की चित्रकारी के असली महत्व में। ...,7,10
105613,we can do to the next harvest,START_ हम लोग अगली फसल तक ये कर सकते है _END,7,11


In [10]:
max_len_encoder = max(df['len_eng_words'])
max_len_decoder = max(df['len_hin_words'])
max_len_encoder , max_len_decoder


(242, 246)

In [11]:
input_words= sorted(list(eng_words))
target_words = sorted(list(hin_words))
num_encoder_tokens = len(input_words) + 1
num_decoder_tokens = len(target_words) + 1
num_encoder_tokens, num_decoder_tokens

(33615, 37770)

In [12]:
input_token_index = dict([(word, i+1) for i,word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i,word in enumerate(target_words)])
reverse_input_token_index = dict([(i, word) for word, i in input_token_index.items()])
reverse_target_token_index = dict([(i, word) for word , i in target_token_index.items()])

In [13]:
df = shuffle(df)
df.head()

,English,Hindi,len_eng_words,len_hin_words
73633,not working or working on average less than ...,START_ और आप काम न कर रहे है या फिर हफ्ते में ...,22,39
114858,from being seen as something forbidding,START_ निषिद्ध रूप से _END,6,5
60617,classhindi,START_ श्रेणीहिन्दी _END,1,3
55092,in the process of identification the more ther...,START_ पहचानने की प्रक्रिया में वहां पर ईर्ष्य...,13,15
66528,afterwards in zimbabwe was suspended from the...,START_ बाद में में जिम्बाब्वे को टेस्ट दर्जे ...,25,32


In [14]:
X, Y = df['English'], df['Hindi']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=43)
X_train.shape, X_test.shape

((24000,), (6000,))

In [15]:
X_train.to_pickle('X_train2Kword.pkl')
X_test.to_pickle('X_test2Kword.pkl')
len(X_train)

24000

In [16]:
def generate_batch(X=X_train, Y=Y_train, batch_size=128):

  while True:

      for j in range(0, len(X), batch_size):
          encoder_input_data = np.zeros((batch_size, max_len_encoder), dtype='float32')
          decoder_input_data = np.zeros((batch_size, max_len_decoder), dtype='float32')
          decoder_target_data= np.zeros((batch_size, max_len_decoder, num_decoder_tokens), dtype='float32')
          for i, (input_text, target_text) in enumerate(zip(X[j : j+batch_size], Y[j : j + batch_size])):
              for t, word in enumerate(input_text.split()):
                  encoder_input_data[i, t] = input_token_index[word]
              for t, word in enumerate(target_text.split()):
                  decoder_input_data[i, t] = target_token_index[word]
                  if t > 0:
                     decoder_target_data[i, t-1 , target_token_index[word]] = 1.
      yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [17]:
#Encoder
latent_dim = 300
encoder_inputs = Input(shape=(None,))
encoder_emb = Embedding(num_decoder_tokens, latent_dim , mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_emb)
encoder_states = [state_h, state_c]



In [18]:
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100

model.fit_generator(generator = generate_batch(X_train, Y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples/batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, Y_test, batch_size = batch_size),
                    validation_steps = val_samples/batch_size)

Epoch 1/100


In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start word.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_token_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
train_gen = generate_batch(X_test, Y_test, batch_size = 1)
k=-1
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', Y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', Y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', Y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])